In [ ]:

import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
#from keras.losses import binary_crossentropy
#from keras.applications.resnet50 import preprocess_input
#import keras.backend as K
#import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score
#from keras.utils import Sequence
#from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = 1000 

In [ ]:
df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

x = df_train['id_code']
y = df_train['diagnosis']


y.hist()

* > ****CROSS VALIDATION TECHNIQUE****

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.15,
                                                      stratify=y, random_state=SEED)
print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape)
train_y.hist()

valid_y.hist()

In [ ]:
from pandas.plotting import scatter_matrix
df = pd.DataFrame(np.random.randn(1000, 4), columns = ['train_x', 'train_y', 'valid_x', 'valid-y'])
scatter_matrix(df, alpha = 0.2, figsize = (6, 6), diagonal = 'kde')

***ORIGINAL INPUT***

In [ ]:
def load_dataset(path):
    eye_files = os.listdir(path)
    return eye_files

In [ ]:
train_a = load_dataset('../input/aptos2019-blindness-detection/train_images')
test_b = load_dataset('../input/aptos2019-blindness-detection/test_images')

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob

train_a = np.array(glob("../input/aptos2019-blindness-detection/train_images/*"))
train_b = np.array(glob("../input/aptos2019-blindness-detection/test_images/*"))
img = cv2.imread(train_a[3])
plt.imshow(img)
plt.show()

In [ ]:
import random
for i in range(10):
    plt.figure(figsize=(10,10))
    i = random.choice(os.listdir('../input/aptos2019-blindness-detection/train_images'))
    i_c = i.split('.')[0]
    img = cv2.imread(os.path.join('../input/aptos2019-blindness-detection/train_images', i))
    
    plt.imshow(img)
    plt.show()

***RGB PREPROCESSING TO INPUTS***

In [ ]:

%%time
import random
fig = plt.figure(figsize=(25, 16))
# display 10 images from each clasS
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (100, 100))

        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

***GRAY SCALE PRE PROCESSING***

In [ ]:
%%time
fig = plt.figure(figsize=(25, 16))
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        image = cv2.resize(image, (100, 100))

        plt.imshow(image, cmap='gray')
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

***ZOOMING THE DETAILS ***
***PRE PROCESSING***

In [ ]:
dpi = 80 #inch

# path=f"../input/aptos2019-blindness-detection/train_images/5c7ab966a3ee.png" # notice upper part
path=f"../input/aptos2019-blindness-detection/train_images/cd54d022e37d.png" # lower-right, this still looks not so severe, can be class3
image = cv2.imread(path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
height, width = image.shape
print(height, width)

SCALE=2
figsize = (width / float(dpi))/SCALE, (height / float(dpi))/SCALE

fig = plt.figure(figsize=figsize)
plt.imshow(image, cmap='gray')

***BEN GRAMS PRE PROCESSING METHOD***

In [ ]:
%%time
fig = plt.figure(figsize=(25, 16))
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (100, 100))
        image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , 100/10) ,-4 ,128) # the trick is to add this line

        plt.imshow(image, cmap='gray')
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

***AUTO CROPPING PRE PROCESSING***

In [ ]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img
    plt.imshow(image, cmap='gray')
    


In [ ]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (100,100))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [ ]:
%%time

NUM_SAMP=7
fig = plt.figure(figsize=(25, 16))
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(NUM_SAMP, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, NUM_SAMP, class_id * NUM_SAMP + i + 1, xticks=[], yticks=[])
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = load_ben_color(path,sigmaX=30)

        plt.imshow(image)
        ax.set_title('%d-%d-%s' % (class_id, idx, row['id_code']) )

***CIRCULAR CROP PRE PROCESSING***

In [ ]:
def circle_crop(img, sigmaX=10):   
    """
    Create circular crop around image centre    
    """    
    
    img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

In [ ]:
%%time
## try circle crop
NUM_SAMP=7
fig = plt.figure(figsize=(25, 16))
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(NUM_SAMP, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, NUM_SAMP, class_id * NUM_SAMP + i + 1, xticks=[], yticks=[])
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = circle_crop(path,sigmaX=30)

        plt.imshow(image)
        ax.set_title('%d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, GaussianNoise, GaussianDropout
from keras.layers import Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.constraints import maxnorm
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import regularizers, optimizers

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
train_df['diagnosis'] = train_df['diagnosis'].astype('str')
train_df['id_code'] = train_df['id_code'].astype(str)+'.png'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

batch_size = 16
image_size = 96



train_gen=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/aptos2019-blindness-detection/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",
    target_size=(image_size,image_size),
    subset='training')

test_gen=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/aptos2019-blindness-detection/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical", 
    target_size=(image_size,image_size),
    subset='validation')

In [ ]:
y_train = train_df['diagnosis']
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
num_classes = y_train.shape[1]

In [ ]:
def build_model():
    # create model
    model = Sequential()
    #model.add(Reshape((x_train.shape[0],),))
    #model.add(GaussianDropout(0.3,input_shape=[96,96,3]))
    model.add(Conv2D(15, (3, 3), input_shape=[96,96,3], activation='relu'))
    model.add(GaussianDropout(0.3))
    model.add(Conv2D(30, (5, 5), activation='relu', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(50, (5, 5), activation='relu'))
    model.add(Conv2D(50, (7, 7), activation='relu'))
    
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.0001)
                   ,activity_regularizer=regularizers.l1(0.01)))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam(lr=0.0001, amsgrad=True), metrics=['accuracy'])
    return model

In [ ]:
model = build_model()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es= EarlyStopping(monitor='val_loss', mode ='min', verbose = 1, patience = 20)
mc = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only = True, mode ='min', verbose = 1)

In [ ]:
model.fit_generator(generator=train_gen,              
                                    steps_per_epoch=len(train_gen),
                                    validation_data=test_gen,                    
                                    validation_steps=len(test_gen),
                                    epochs=1,
                                    callbacks = [es, mc], 
                                    use_multiprocessing = True,
                                    verbose=1)


In [ ]:
from keras.models import load_model
model = load_model('model.h5')

In [ ]:
submission_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
#submission_df['diagnosis'] = submission_df['diagnosis'].astype('str')
submission_df['filename'] = submission_df['id_code'].astype(str)+'.png'

In [ ]:
submission_datagen=ImageDataGenerator(rescale=1./255)
submission_gen=submission_datagen.flow_from_dataframe(
    dataframe=submission_df,
    directory="../input/aptos2019-blindness-detection/test_images",
    x_col="filename",    
    batch_size=batch_size,
    shuffle=False,
    class_mode=None, 
    target_size=(image_size,image_size)
)

In [ ]:
predictions=model.predict_generator(submission_gen, steps = len(submission_gen))

In [ ]:
max_probability = np.argmax(predictions,axis=1)

In [ ]:
submission_df.drop(columns=['filename'], inplace= True)
submission_df['diagnosis'] = max_probability
submission_df.to_csv('submission.csv', index=False)

**random forest**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import time

#model selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  

# to make this notebook's output stable across runs
np.random.seed(42)

In [ ]:
IMG_SIZE = 128
DATA_DIR = os.path.join(r'/kaggle/input/', 'aptos2019-blindness-detection')
TRAIN_DIR = os.path.join(DATA_DIR,'train_images')
TEST_DIR = os.path.join(DATA_DIR, 'test_images')

def searchCircle(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    rows = gray.shape[0]
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, rows / 8,
                               param1=100, param2=30,
                               minRadius=int(img.shape[1]/4), maxRadius=int(img.shape[1]/2))
    
    if circles is not None:
        circles = np.int32(np.around(circles))
        return circles[0,0]
    
    cx = int(img.shape[1]/2)
    cy = int(img.shape[0]/2)
    r = max(cx, cy)
    return np.array([cx, cy, r])

def circle_crop(img, cx, cy, r):
    mask = np.zeros((img.shape[1], img.shape[0]), np.uint8)
    cv2.circle(mask, (cx, cy), r, (255,255,255), thickness=-1)
    return cv2.bitwise_and(img, img, mask=mask)

def rect_crop(img, x0, y0, x1, y1):
    mask = np.zeros((img.shape[1], img.shape[0]), np.uint8)
    cv2.rectangle(mask, (x0, y0), (x1, y1), (255,255,255), thickness=-1)
    return cv2.bitwise_and(img, img, mask=mask)

def processImg(img):
    # find the circle and resize
    cir = searchCircle(img)     
    scale = (float(IMG_SIZE)) / (cir[2] * 2)
    img = cv2.resize(img,(0,0),fx=scale, fy=scale)
    
    cir = np.int32(np.around(cir * scale))
    
    # crop the circle
    x0 = max(cir[0]-cir[2], 0)
    x1 = min(cir[0]+cir[2], img.shape[1])
    y0 = max(cir[1]-cir[2], 0)
    y1 = min(cir[1]+cir[2], img.shape[0])
    img = img[y0:y1, x0:x1,:]
    
    # copy the circle to the center of square
    img1 = np.zeros([cir[2] * 2, cir[2] * 2, img.shape[2]])
    dx = max(cir[2] - cir[0], 0)
    dy = max(cir[2] - cir[1], 0)
    img1[dy:dy+y1-y0,dx:dx+x1-x0,:] = img
    
    # crop the cirle
    r = int(IMG_SIZE/2)
    img1 = circle_crop(img1, r, r, r)
    img1 = rect_crop(img1, dx, dy, dx+x1-x0, dy+y1-y0)

    img1 = np.array(img1)/255 # normalize the image
    return img1

def getImageData(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)

    if img is not None:
        return processImg(img)
    
    return None

def idcode2Path(imgDir, idCode):
    return os.path.join(imgDir, '{}.png'.format(idCode))

def getImgArray(idList, imgDir):
    imgs = []
    for i, idCode in enumerate(idList):
        img = getImageData(idcode2Path(imgDir, idCode))
        if img is None:
            print(idCode , " is none")
        imgs.append(img)
        
    return np.array(imgs)

def showOne(ax, imgs, index):
    if(index >= len(imgs)): 
        return False
    
    img1 = imgs[index].reshape(IMG_SIZE, IMG_SIZE, 3)
    ax.imshow(img1)
    ax.axis("off")
    
    return True

def plotMatrix(funcOne, paramList, row=1, col=3, figsize=None):
    if figsize is None:
        figsize=(col * 5, row * 4)
    fig, axes = plt.subplots(row, col,figsize=figsize)
    if((row > 1) | (col > 1)):
        axes = axes.ravel()
    else:
        axes = [axes]
        
    for i, ax in enumerate(axes):
        if(not funcOne(ax, paramList, i)): 
            break
   
    plt.show()

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
print(df.shape)

In [ ]:
t0 = time.time()
X = getImgArray(df['id_code'], TRAIN_DIR)
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2]*X.shape[3])
Y = df['diagnosis']

print("getImgArray elapsed time:", time.time() - t0)

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
model = RandomForestClassifier(random_state=42, bootstrap=False, criterion='entropy', 
           max_features=10,min_samples_split=10, max_depth=38, n_estimators=27)

model.fit(x_train, y_train)

y_valid_pred = model.predict(x_valid)

score = accuracy_score(y_valid, y_valid_pred)
print("Valid Score = {0:.4f}".format(score))

In [ ]:
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))
x_test = getImgArray(test_df['id_code'], TEST_DIR)

print(x_test.shape)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]*x_test.shape[3])

In [ ]:
model.fit(X, np.array(Y))
pred = model.predict(x_test)

test_df['diagnosis'] = pred
test_df.to_csv('submission2.csv',index=False)

**KNN BEGINS**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix 
from sklearn.metrics import cohen_kappa_score
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

#Ensures consistency across runs
from numpy.random import seed
seed(1)

#Imports to view data
import cv2
from glob import glob
import matplotlib.pyplot as plt
from numpy import floor
import random

#others
import os
print(os.listdir('../input/'))

In [ ]:
train_df=pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df=pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
print('Size of train dataset',train_df.shape)
print('Size of test dataset',test_df.shape)

In [ ]:
train_labels=train_df['diagnosis']
display(train_labels.head())

In [ ]:
target_classes=['No DR','Mild','Moderate','Sever','Proliferative DR']

In [ ]:
from sklearn.utils import shuffle
id_code=train_df['id_code']
diagnosis=train_df['diagnosis']
id_code,diagnosis=shuffle(id_code,diagnosis,random_state=42)
plt.figure(figsize=(15,8))
plt.xlabel('diagnosis')
plt.ylabel('count')
diagnosis.hist()

In [ ]:
def read_image(path,id_code,size):
    img_path=os.path.join(path,id_code+'.png')
    image=cv2.imread(img_path)
    image=cv2.resize(image,(size,size))
    #Normalizing pixel data (0-255)
    image=image.reshape(size,size,3).astype('float32')/255  
    return image

In [ ]:
#Build train images data as a numpy array
train_images=[]
train_images.append(train_df['id_code'].apply(lambda x:read_image
                                              ('../input/aptos2019-blindness-detection/train_images',x,128)))
df_train=np.array(train_images)
df_train=df_train.reshape(df_train.shape[1],128,128,3).astype('float32')
print(df_train.shape)


In [ ]:
test_df=pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_images=[]
test_images.append(test_df['id_code'].apply(lambda x:read_image
                                              ('../input/aptos2019-blindness-detection/test_images',x,128)))
X_test=np.array(test_images)
X_test=X_test.reshape(X_test.shape[1],128,128,3).astype('float32')
print(X_test.shape)

In [ ]:
df_train_2d = len(df_train)
df_train= df_train.reshape(df_train_2d,-1)

In [ ]:
print('Before sampling...')
print('Size of the train dataset:',len(df_train))
print("Before sampling,counts of label '0':{}".format(sum(train_labels==0)))
print("Before sampling,counts of label '1':{}".format(sum(train_labels==1)))
print("Before sampling,counts of label '2':{}".format(sum(train_labels==2)))
print("Before sampling,counts of label '3':{}".format(sum(train_labels==3)))
print("Before sampling,counts of label '4':{}".format(sum(train_labels==4)))
#Apply SMOTE technique
sm=SMOTE(random_state=42,k_neighbors=3)
X_train_res,Y_train_res=sm.fit_sample(df_train,train_labels.ravel())

#After sampling
print('After sampling...')
print('Size of the train dataset:',len(X_train_res))
print("Before sampling,counts of label '0':{}".format(sum(Y_train_res==0)))
print("Before sampling,counts of label '1':{}".format(sum(Y_train_res==1)))
print("Before sampling,counts of label '2':{}".format(sum(Y_train_res==2)))
print("Before sampling,counts of label '3':{}".format(sum(Y_train_res==3)))
print("Before sampling,counts of label '4':{}".format(sum(Y_train_res==4)))

In [ ]:
from sklearn.model_selection import train_test_split
X_train1,X_valid,Y_train1,Y_valid=train_test_split(X_train_res,Y_train_res,
                                                 test_size=0.1,
                                                 random_state=42)
print(len(X_train1),len(X_valid))
print(len(Y_train1),len(Y_valid))

In [ ]:
#convert 3d array to 1d array
X_test_2d = len(X_test)
X_test= X_test.reshape(X_test_2d,-1)
print(X_train1.shape,X_valid.shape,X_test.shape)

In [ ]:
print('Evaluating the KNN Classifier...')
model=KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
model.fit(X_train1,Y_train1)
Y_pred=model.predict(X_valid)
classification_report=classification_report(Y_valid,Y_pred,target_names=target_classes)
display(classification_report)
confusion_matrix=confusion_matrix(Y_valid,Y_pred)
display(confusion_matrix)
kappa_score=cohen_kappa_score(Y_valid,Y_pred,weights='quadratic')
print('Quadratic Kappa Score:')
display(kappa_score)

In [ ]:
#prediction on unseen data (test data)
KNN=pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
Y_predict=model.predict(X_test)
KNN['diagnosis']=Y_predict
KNN.to_csv('submission3.csv',index=False)